<a href="https://colab.research.google.com/github/enderunknown222/MA/blob/Ver0.1/MA4550_Project_ver0.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import io
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
pd.options.mode.chained_assignment = None

In [4]:
from google.colab import files
uploaded = files.upload()
data = pd.read_csv(io.BytesIO(uploaded['data.csv']))

Saving data.csv to data.csv


In [5]:
#test data size
x=190

Part 1. Predict without any stats

In [5]:
#Change name to ID
results = data.iloc[:,1:7]
Name=['Arsenal','Aston Villa','Brighton','Burnley','Chelsea','Crystal Palace','Everton','Fulham','Leeds United','Leicester City','Liverpool','Manchester City','Manchester Utd','Newcastle Utd','Sheffield Utd','Southampton','Tottenham','West Brom','West Ham','Wolves']
for i in range(380):
  for j in ['Home','Away']:
    for k in range(20):
      if results[j][i]==Name[k]:
        results[j][i]=k
results

,Home,HxG,HScore,AScore,AxG,Away
0,7,0.2,0.0,3.0,1.8,0
1,5,0.7,1.0,0.0,0.8,15
2,10,3.3,4.0,3.0,0.6,8
3,18,1.1,0.0,2.0,1.5,13
4,17,0.5,0.0,3.0,2.2,9
...,...,...,...,...,...,...
375,0,NaN,NaN,NaN,NaN,2
376,18,NaN,NaN,NaN,NaN,15
377,14,NaN,NaN,NaN,NaN,3
378,10,NaN,NaN,NaN,NaN,5


In [12]:
traindata=results.iloc[0:x,:]
preddata=results.iloc[x:380,:]
newresults=results
x_train = traindata[['Home', 'Away']]
y_train = traindata[['HScore', 'AScore']]
x_pred = preddata[['Home', 'Away']]
newresults

,Home,HxG,HScore,AScore,AxG,Away
0,7,0.2,0.000000,3.0,1.8,0
1,5,0.7,1.000000,0.0,0.8,15
2,10,3.3,4.000000,3.0,0.6,8
3,18,1.1,0.000000,2.0,1.5,13
4,17,0.5,0.000000,3.0,2.2,9
...,...,...,...,...,...,...
375,0,NaN,0.333333,NaN,NaN,2
376,18,NaN,0.707107,NaN,NaN,15
377,14,NaN,1.242641,NaN,NaN,3
378,10,NaN,2.242641,NaN,NaN,5


In [7]:
model = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3, weights='distance')).fit(x_train, y_train)

In [8]:
goals = model.predict(x_pred)

In [9]:
for i in range(380-x):
  for j in range(1):
    if j==0:
      k='HScore'
    if j==1:
      k='AScore'
    newresults[k][x+i]=goals[i][j]
newresults

,Home,HxG,HScore,AScore,AxG,Away
0,7,0.2,0.000000,3.0,1.8,0
1,5,0.7,1.000000,0.0,0.8,15
2,10,3.3,4.000000,3.0,0.6,8
3,18,1.1,0.000000,2.0,1.5,13
4,17,0.5,0.000000,3.0,2.2,9
...,...,...,...,...,...,...
375,0,NaN,0.333333,NaN,NaN,2
376,18,NaN,0.707107,NaN,NaN,15
377,14,NaN,1.242641,NaN,NaN,3
378,10,NaN,2.242641,NaN,NaN,5


In [13]:
lb = {'Team': Name,
    'Win':np.zeros((20)),
    'Loss':np.zeros((20)),
    'Draw':np.zeros((20)),
    'Pts':np.zeros((20))
      }
leaderboard = pd.DataFrame(lb, columns= ['Team','Win','Loss','Draw','Pts'])
for i in range(380):
  if newresults['HScore'][i]>newresults['AScore'][i]:
    leaderboard['Win'][newresults['Home'][i]]+=1
    leaderboard['Pts'][newresults['Home'][i]]+=3
    leaderboard['Loss'][newresults['Away'][i]]+=1
    leaderboard['Pts'][newresults['Away'][i]]+=0
  if newresults['HScore'][i]==newresults['AScore'][i]:
    leaderboard['Draw'][newresults['Home'][i]]+=1
    leaderboard['Pts'][newresults['Home'][i]]+=1
    leaderboard['Draw'][newresults['Away'][i]]+=1
    leaderboard['Pts'][newresults['Away'][i]]+=1
  if newresults['HScore'][i]<newresults['AScore'][i]:
    leaderboard['Loss'][newresults['Home'][i]]+=1
    leaderboard['Pts'][newresults['Home'][i]]+=0
    leaderboard['Win'][newresults['Away'][i]]+=1
    leaderboard['Pts'][newresults['Away'][i]]+=3

In [17]:
leaderboard.sort_values(by=['Pts','Win'],inplace=True, ascending=False)
leaderboard

,Team,Win,Loss,Draw,Pts
11,Manchester City,21.0,5.0,5.0,68.0
12,Manchester Utd,17.0,8.0,5.0,56.0
10,Liverpool,16.0,6.0,8.0,56.0
16,Tottenham,15.0,8.0,7.0,52.0
0,Arsenal,15.0,11.0,4.0,49.0
6,Everton,15.0,10.0,4.0,49.0
8,Leeds United,15.0,12.0,3.0,48.0
9,Leicester City,15.0,12.0,3.0,48.0
18,West Ham,14.0,10.0,6.0,48.0
4,Chelsea,14.0,11.0,5.0,47.0


Part2: Using xG to predict the goal efficiency of each team in each match

In [18]:
#Change name to ID
results2 = data.iloc[:,1:7]
Name=['Arsenal','Aston Villa','Brighton','Burnley','Chelsea','Crystal Palace','Everton','Fulham','Leeds United','Leicester City','Liverpool','Manchester City','Manchester Utd','Newcastle Utd','Sheffield Utd','Southampton','Tottenham','West Brom','West Ham','Wolves']
for i in range(380):
  for j in ['Home','Away']:
    for k in range(20):
      if results2[j][i]==Name[k]:
        results2[j][i]=k
results2.insert(1,'HGoalEff',np.zeros(380), True)
results2.insert(6,'AGoalEff',np.zeros(380), True)
results2

,Home,HGoalEff,HxG,HScore,AScore,AxG,AGoalEff,Away
0,7,0.0,0.2,0.0,3.0,1.8,0.0,0
1,5,0.0,0.7,1.0,0.0,0.8,0.0,15
2,10,0.0,3.3,4.0,3.0,0.6,0.0,8
3,18,0.0,1.1,0.0,2.0,1.5,0.0,13
4,17,0.0,0.5,0.0,3.0,2.2,0.0,9
...,...,...,...,...,...,...,...,...
375,0,0.0,NaN,NaN,NaN,NaN,0.0,2
376,18,0.0,NaN,NaN,NaN,NaN,0.0,15
377,14,0.0,NaN,NaN,NaN,NaN,0.0,3
378,10,0.0,NaN,NaN,NaN,NaN,0.0,5


In [19]:
GoalEff=[np.arange(20),np.zeros(20),np.zeros(20),np.zeros(20),np.zeros(20)]
for i in range(x):
  if results2['HScore'][i]==0:
    GoalEff[1][results2['Home'][i]]+=(results2['HScore'][i]-results2['HxG'][i])/0.5
  else:
    GoalEff[1][results2['Home'][i]]+=(results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]
  GoalEff[2][results2['Home'][i]]+=1
  if results2['AScore'][i]==0:
    GoalEff[1][results2['Away'][i]]+=(results2['AScore'][i]-results2['AxG'][i])/0.5
  else:
    GoalEff[1][results2['Away'][i]]+=(results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]
  GoalEff[2][results2['Away'][i]]+=1
for i in range(20):
  GoalEff[3][i]=GoalEff[1][i]/GoalEff[2][i]
for i in range(x):
  if results2['HScore'][i]==0:
    GoalEff[4][results2['Home'][i]]+=((results2['HScore'][i]-results2['HxG'][i])/0.5-GoalEff[3][results2['Home'][i]])**2
  else:
    GoalEff[4][results2['Home'][i]]+=((results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]-GoalEff[3][results2['Home'][i]])**2
  if results2['AScore'][i]==0:
    GoalEff[4][results2['Away'][i]]+=((results2['AScore'][i]-results2['AxG'][i])/0.5-GoalEff[3][results2['Away'][i]])**2
  else:
    GoalEff[4][results2['Away'][i]]+=((results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]-GoalEff[3][results2['Away'][i]])**2
for i in range(20):
  GoalEff[4][i]=GoalEff[4][i]/GoalEff[2][i]


In [20]:
for i in range(x):
  if results2['HScore'][i]==0:
    results2['HGoalEff'][i]=(results2['HScore'][i]-results2['HxG'][i])/0.5
  else:
    results2['HGoalEff'][i]=(results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]
  if results2['AScore'][i]==0:
    results2['AGoalEff'][i]=(results2['AScore'][i]-results2['AxG'][i])/0.5
  else:
    results2['AGoalEff'][i]=(results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]
for i in range(380-x):
  results2['HGoalEff'][x+i]=np.random.normal(GoalEff[3][results2['Home'][x+i]],GoalEff[4][results2['Home'][x+i]])
  results2['AGoalEff'][x+i]=np.random.normal(GoalEff[3][results2['Away'][x+i]],GoalEff[4][results2['Away'][x+i]])
results2

,Home,HGoalEff,HxG,HScore,AScore,AxG,AGoalEff,Away
0,7,-0.400000,0.2,0.0,3.0,1.8,0.400000,0
1,5,0.300000,0.7,1.0,0.0,0.8,-1.600000,15
2,10,0.175000,3.3,4.0,3.0,0.6,0.800000,8
3,18,-2.200000,1.1,0.0,2.0,1.5,0.250000,13
4,17,-1.000000,0.5,0.0,3.0,2.2,0.266667,9
...,...,...,...,...,...,...,...,...
375,0,0.562537,NaN,NaN,NaN,NaN,-1.411760,2
376,18,-0.190053,NaN,NaN,NaN,NaN,-0.599188,15
377,14,-0.065958,NaN,NaN,NaN,NaN,-0.989702,3
378,10,1.360968,NaN,NaN,NaN,NaN,0.754000,5


In [21]:
traindata2=results2.iloc[0:x,:]
preddata2=results2.iloc[x:380,:]
newresults2=results2
x_train2 = traindata2[['Home','HGoalEff','AGoalEff','Away']]
y_train2 = traindata2[['HScore', 'AScore']]
x_pred2 = preddata2[['Home','HGoalEff','AGoalEff','Away']]

In [22]:
model2 = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3, weights='distance')).fit(x_train2, y_train2)

In [23]:
goals2 = model2.predict(x_pred2)

In [24]:
for i in range(380-x):
  for j in range(2):
    if j==0:
      k='HScore'
    if j==1:
      k='AScore'
    newresults2[k][x+i]=goals2[i][j]
newresults2

,Home,HGoalEff,HxG,HScore,AScore,AxG,AGoalEff,Away
0,7,-0.400000,0.2,0.000000,3.000000,1.8,0.400000,0
1,5,0.300000,0.7,1.000000,0.000000,0.8,-1.600000,15
2,10,0.175000,3.3,4.000000,3.000000,0.6,0.800000,8
3,18,-2.200000,1.1,0.000000,2.000000,1.5,0.250000,13
4,17,-1.000000,0.5,0.000000,3.000000,2.2,0.266667,9
...,...,...,...,...,...,...,...,...
375,0,0.562537,NaN,2.290092,1.111822,NaN,-1.411760,2
376,18,-0.190053,NaN,0.729323,1.000000,NaN,-0.599188,15
377,14,-0.065958,NaN,1.154941,1.917897,NaN,-0.989702,3
378,10,1.360968,NaN,3.214885,1.317857,NaN,0.754000,5


In [25]:
lb = {'Team': Name,
    'Win':np.zeros((20)),
    'Loss':np.zeros((20)),
    'Draw':np.zeros((20)),
    'Pts':np.zeros((20))
      }
leaderboard2 = pd.DataFrame(lb, columns= ['Team','Win','Loss','Draw','Pts'])
for i in range(380):
  if newresults2['HScore'][i]>newresults2['AScore'][i]:
    leaderboard2['Win'][newresults2['Home'][i]]+=1
    leaderboard2['Pts'][newresults2['Home'][i]]+=3
    leaderboard2['Loss'][newresults2['Away'][i]]+=1
    leaderboard2['Pts'][newresults2['Away'][i]]+=0
  if newresults2['HScore'][i]==newresults2['AScore'][i]:
    leaderboard2['Draw'][newresults2['Home'][i]]+=1
    leaderboard2['Pts'][newresults2['Home'][i]]+=1
    leaderboard2['Draw'][newresults2['Away'][i]]+=1
    leaderboard2['Pts'][newresults2['Away'][i]]+=1
  if newresults2['HScore'][i]<newresults2['AScore'][i]:
    leaderboard2['Loss'][newresults2['Home'][i]]+=1
    leaderboard2['Pts'][newresults2['Home'][i]]+=0
    leaderboard2['Win'][newresults2['Away'][i]]+=1
    leaderboard2['Pts'][newresults2['Away'][i]]+=3

In [26]:
leaderboard2.sort_values(by=['Pts','Win'],inplace=True, ascending=False)
leaderboard2

,Team,Win,Loss,Draw,Pts
10,Liverpool,25.0,6.0,7.0,82.0
11,Manchester City,24.0,8.0,6.0,78.0
12,Manchester Utd,24.0,10.0,4.0,76.0
0,Arsenal,22.0,13.0,3.0,69.0
9,Leicester City,22.0,13.0,3.0,69.0
1,Aston Villa,22.0,14.0,2.0,68.0
15,Southampton,20.0,13.0,5.0,65.0
6,Everton,19.0,17.0,2.0,59.0
4,Chelsea,18.0,15.0,5.0,59.0
16,Tottenham,17.0,15.0,6.0,57.0


Repeat Part 2 1000 times to find the expected win percentage

In [36]:
y=1000
results2 = data.iloc[:,1:7]
Name=['Arsenal','Aston Villa','Brighton','Burnley','Chelsea','Crystal Palace','Everton','Fulham','Leeds United','Leicester City','Liverpool','Manchester City','Manchester Utd','Newcastle Utd','Sheffield Utd','Southampton','Tottenham','West Brom','West Ham','Wolves']
for i in range(380):
  for j in ['Home','Away']:
    for k in range(20):
      if results2[j][i]==Name[k]:
        results2[j][i]=k
results2.insert(1,'HGoalEff',np.zeros(380), True)
results2.insert(6,'AGoalEff',np.zeros(380), True)
GoalEff=[np.arange(20),np.zeros(20),np.zeros(20),np.zeros(20),np.zeros(20)]  
aclb = {'Team': Name,
  'Champ':np.zeros(20),
  'Champ%':np.zeros(20),
  'Top4':np.zeros(20),
  'Top4%':np.zeros(20)
    }  
for i in range(x):
  if results2['HScore'][i]==0:
    GoalEff[1][results2['Home'][i]]+=(results2['HScore'][i]-results2['HxG'][i])/0.5
  else:
    GoalEff[1][results2['Home'][i]]+=(results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]
  GoalEff[2][results2['Home'][i]]+=1
  if results2['AScore'][i]==0:
    GoalEff[1][results2['Away'][i]]+=(results2['AScore'][i]-results2['AxG'][i])/0.5
  else:
    GoalEff[1][results2['Away'][i]]+=(results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]
  GoalEff[2][results2['Away'][i]]+=1
for i in range(20):
  GoalEff[3][i]=GoalEff[1][i]/GoalEff[2][i]
for i in range(x):
  if results2['HScore'][i]==0:
    GoalEff[4][results2['Home'][i]]+=((results2['HScore'][i]-results2['HxG'][i])/0.5-GoalEff[3][results2['Home'][i]])**2
  else:
    GoalEff[4][results2['Home'][i]]+=((results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]-GoalEff[3][results2['Home'][i]])**2
  if results2['AScore'][i]==0:
    GoalEff[4][results2['Away'][i]]+=((results2['AScore'][i]-results2['AxG'][i])/0.5-GoalEff[3][results2['Away'][i]])**2
  else:
    GoalEff[4][results2['Away'][i]]+=((results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]-GoalEff[3][results2['Away'][i]])**2
for i in range(20):
  GoalEff[4][i]=GoalEff[4][i]/GoalEff[2][i]
for i in range(x):
  if results2['HScore'][i]==0:
    results2['HGoalEff'][i]=(results2['HScore'][i]-results2['HxG'][i])/0.5
  else:
    results2['HGoalEff'][i]=(results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]
  if results2['AScore'][i]==0:
    results2['AGoalEff'][i]=(results2['AScore'][i]-results2['AxG'][i])/0.5
  else:
    results2['AGoalEff'][i]=(results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]
for a in range(y):
    for i in range(380-x):
      results2['HGoalEff'][x+i]=np.random.normal(GoalEff[3][results2['Home'][x+i]],GoalEff[4][results2['Home'][x+i]])
      results2['AGoalEff'][x+i]=np.random.normal(GoalEff[3][results2['Away'][x+i]],GoalEff[4][results2['Away'][x+i]])
    traindata2=results2.iloc[0:x,:]
    preddata2=results2.iloc[x:380,:]
    newresults2=results2
    x_train2 = traindata2[['Home','HGoalEff','AGoalEff','Away']]
    y_train2 = traindata2[['HScore', 'AScore']]
    x_pred2 = preddata2[['Home','HGoalEff','AGoalEff','Away']]
    model2 = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3, weights='distance')).fit(x_train2, y_train2)
    goals2 = model2.predict(x_pred2)
    for i in range(380-x):
      for j in range(2):
        if j==0:
          k='HScore'
        if j==1:
          k='AScore'
        newresults2[k][x+i]=goals2[i][j]
    lb = {'Team': np.arange(20),
    'Win':np.zeros((20)),
    'Loss':np.zeros((20)),
    'Draw':np.zeros((20)),
    'Pts':np.zeros((20))
      }
    leaderboard2 = pd.DataFrame(lb, columns= ['Team','Win','Loss','Draw','Pts']) 
    for i in range(380):
      if newresults2['HScore'][i]>newresults2['AScore'][i]:
        leaderboard2['Win'][newresults2['Home'][i]]+=1
        leaderboard2['Pts'][newresults2['Home'][i]]+=3
        leaderboard2['Loss'][newresults2['Away'][i]]+=1
        leaderboard2['Pts'][newresults2['Away'][i]]+=0
      if newresults2['HScore'][i]==newresults2['AScore'][i]:
        leaderboard2['Draw'][newresults2['Home'][i]]+=1
        leaderboard2['Pts'][newresults2['Home'][i]]+=1
        leaderboard2['Draw'][newresults2['Away'][i]]+=1
        leaderboard2['Pts'][newresults2['Away'][i]]+=1
      if newresults2['HScore'][i]<newresults2['AScore'][i]:
        leaderboard2['Loss'][newresults2['Home'][i]]+=1
        leaderboard2['Pts'][newresults2['Home'][i]]+=0
        leaderboard2['Win'][newresults2['Away'][i]]+=1
        leaderboard2['Pts'][newresults2['Away'][i]]+=3
    c=leaderboard2.loc[leaderboard2['Pts'].idxmax(), 'Team']
    aclb['Champ'][c]+=1
    for l in range(4):
      d=leaderboard2.nlargest(4,'Pts')['Team'].iloc[l]
      aclb['Top4'][d]+=1
accleaderboard = pd.DataFrame(aclb, columns= ['Team','Champ','Win%','Top4','Top4%'])
accleaderboard['Win%']=accleaderboard['Champ']/y*100
accleaderboard['Top4%']=accleaderboard['Top4']/y*100
accleaderboard.sort_values(by=['Champ','Top4'],inplace=True, ascending=False)
accleaderboard

,Team,Champ,Win%,Top4,Top4%
11,Manchester City,617.0,61.7,993.0,99.3
12,Manchester Utd,178.0,17.8,905.0,90.5
10,Liverpool,130.0,13.0,767.0,76.7
9,Leicester City,63.0,6.3,623.0,62.3
0,Arsenal,5.0,0.5,210.0,21.0
6,Everton,4.0,0.4,228.0,22.8
16,Tottenham,1.0,0.1,95.0,9.5
4,Chelsea,1.0,0.1,63.0,6.3
15,Southampton,1.0,0.1,56.0,5.6
1,Aston Villa,0.0,0.0,45.0,4.5


In [34]:
leaderboard2.nlargest(4,'Pts')['Team'].iloc[2]

,Team,Win,Loss,Draw,Pts
0,0,18.0,17.0,3.0,57.0
1,1,20.0,16.0,2.0,62.0
2,2,12.0,18.0,8.0,44.0
3,3,15.0,19.0,4.0,49.0
4,4,20.0,13.0,5.0,65.0
5,5,20.0,13.0,5.0,65.0
6,6,22.0,14.0,2.0,68.0
7,7,11.0,21.0,6.0,39.0
8,8,12.0,24.0,2.0,38.0
9,9,21.0,15.0,2.0,65.0
